# LangChain Expression Language (LCEL)

# RunnableLambda

In [23]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

def some_func_x(x):
    return x

def some_func_y(y):
    return 2*y

def some_func_z(z):
    return z

runnable_func_x = RunnableLambda(some_func_x)
print(runnable_func_x.invoke("x"))
# print(runnable_func_x.invoke({"x": "x"}))
# print(runnable_func_x.invoke({"x"}))
# print(runnable_func_x.invoke(input="x"))


chain = RunnableLambda(some_func_x) | RunnableLambda(some_func_y)
print(chain.invoke("x"))
chain = RunnableLambda(some_func_x) | some_func_y
print(chain.invoke("x"))
chain = some_func_x | RunnableLambda(some_func_y)
print(chain.invoke("x"))

chain = RunnableParallel({"x": RunnableLambda(some_func_x), "y": RunnableLambda(some_func_y)})
print(chain.invoke("x"))

chain = RunnableParallel({"x": RunnableLambda(some_func_x), "y": RunnableLambda(some_func_y)}) | RunnableLambda(some_func_z)
print(chain.invoke("x"))

chain = {"x": RunnableLambda(some_func_x), "y": RunnableLambda(some_func_y)} | RunnableLambda(some_func_z)
print(chain.invoke("x"))

chain = {"x": some_func_x, "y": some_func_y} | RunnableLambda(some_func_z)
print(chain.invoke("x"))

x
xx
xx
xx
{'x': 'x', 'y': 'xx'}
{'x': 'x', 'y': 'xx'}
{'x': 'x', 'y': 'xx'}
{'x': 'x', 'y': 'xx'}
